## Predicting STEAM User Engagement

### Statement Problem

Customer engagement is a measurement of a user's response to the product. By prediciting STEAM video games and user engagement with a predicitive model, I hope to improve user experience and customer engagement to contribute to the success of video games on STEAM. 

### Data

Datasets in data folder are all provided by KAGGLE

In [1]:
import pandas as pd
import os
import glob
import numpy as np

### CLEANING steam_games1

In [2]:
spy = pd.read_csv('./data/data_raw/steamspy_data.csv')
steam = pd.read_csv('./data/data_raw/steam.csv')
k_200 = pd.read_csv('./data/data_raw/steam-200k.csv')
games = pd.read_csv('./data/data_raw/steam_games.csv')
app = pd.read_csv('./data/data_raw/steam_app_data.csv')

In [3]:
# print out number of rows and columns
print('Spy Shape:', spy.shape)
print('App Shape:', app.shape)
print('Steam Shape:', steam.shape)
print('k_200 Shape:', k_200.shape)
print('Games Shape:', games.shape)

Spy Shape: (29235, 20)
App Shape: (29235, 39)
Steam Shape: (27075, 18)
k_200 Shape: (199999, 5)
Games Shape: (32711, 10)


In [4]:
app.head(2)

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,...,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Counter-Strike,10,0.0,False,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,...,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 65735},{'total': 0},"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,game,Team Fortress Classic,20,0.0,False,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,...,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 2802},{'total': 0},"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."


In [5]:
null_values = app.isnull().sum()

In [6]:
threshold = len(app) * .4
drop_rows = app.columns[null_values > threshold]

print('Columns to drop: {}'.format(list(drop_rows)))

Columns to drop: ['controller_support', 'dlc', 'fullgame', 'legal_notice', 'drm_notice', 'ext_user_account_notice', 'demos', 'metacritic', 'reviews', 'recommendations']


In [7]:
app.drop(columns=['controller_support', 'dlc', 'fullgame', 'legal_notice', 'drm_notice', 'ext_user_account_notice', 'demos', 'metacritic', 'reviews', 'recommendations'], inplace=True)

In [8]:
app.dtypes

type                     object
name                     object
steam_appid               int64
required_age            float64
is_free                  object
detailed_description     object
about_the_game           object
short_description        object
supported_languages      object
header_image             object
website                  object
pc_requirements          object
mac_requirements         object
linux_requirements       object
developers               object
publishers               object
price_overview           object
packages                 object
package_groups           object
platforms                object
categories               object
genres                   object
screenshots              object
movies                   object
achievements             object
release_date             object
support_info             object
background               object
content_descriptors      object
dtype: object

In [9]:
# Check for duplicated rows
duplicated_rows = app[app.duplicated()]

print('Duplicate rows to remove:', duplicated_rows.shape[0])

duplicated_rows.head(2)

Duplicate rows to remove: 7


,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,...,platforms,categories,genres,screenshots,movies,achievements,release_date,support_info,background,content_descriptors
31,game,SiN Episodes: Emergence,1300,0.0,False,"You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","English, Russian, French",https://steamcdn-a.akamaihd.net/steam/apps/130...,...,"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '10 May, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/130...,"{'ids': [], 'notes': None}"
32,game,SiN Episodes: Emergence,1300,0.0,False,"You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","You are John Blade, commander of HardCorps, an...","English, Russian, French",https://steamcdn-a.akamaihd.net/steam/apps/130...,...,"{'windows': True, 'mac': False, 'linux': False}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 0},"{'coming_soon': False, 'date': '10 May, 2006'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/130...,"{'ids': [], 'notes': None}"


In [10]:
app = app.drop_duplicates()

In [11]:
app.drop(columns=['header_image', 'website', 'movies', 'achievements', 'support_info', 'background', 'content_descriptors','price_overview', 'packages'], inplace=True)

In [12]:
app.isnull().sum()

type                    149
name                      1
steam_appid               0
required_age            149
is_free                 149
detailed_description    175
about_the_game          175
short_description       175
supported_languages     163
pc_requirements         149
mac_requirements        149
linux_requirements      149
developers              264
publishers              149
package_groups          149
platforms               149
categories              714
genres                  196
screenshots             177
release_date            149
dtype: int64

In [13]:
app = app.rename(columns={'steam_appid':'appid'})

In [14]:
spy.head(2)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,10,Counter-Strike,Valve,Valve,NaN,124534,3339,0,"10,000,000 .. 20,000,000",17612,709,317,26,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,14923,"{'Action': 2681, 'FPS': 2048, 'Multiplayer': 1..."
1,20,Team Fortress Classic,Valve,Valve,NaN,3318,633,0,"5,000,000 .. 10,000,000",277,15,62,15,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,87,"{'Action': 208, 'FPS': 188, 'Multiplayer': 172..."


In [15]:
null_values_spy = spy.isnull().sum()

In [16]:
threshold_spy = len(spy) * .4
drop_rows_spy = spy.columns[null_values_spy > threshold]

print('Columns to drop: {}'.format(list(drop_rows_spy)))

Columns to drop: ['score_rank']


In [17]:
spy = spy.drop(columns='score_rank')

In [18]:
merged = app.merge(spy, on='appid')

In [19]:
merged.shape

(29227, 38)

In [20]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29227 entries, 0 to 29226
Data columns (total 38 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  29078 non-null  object 
 1   name_x                29226 non-null  object 
 2   appid                 29227 non-null  int64  
 3   required_age          29078 non-null  float64
 4   is_free               29078 non-null  object 
 5   detailed_description  29052 non-null  object 
 6   about_the_game        29052 non-null  object 
 7   short_description     29052 non-null  object 
 8   supported_languages   29064 non-null  object 
 9   pc_requirements       29078 non-null  object 
 10  mac_requirements      29078 non-null  object 
 11  linux_requirements    29078 non-null  object 
 12  developers            28963 non-null  object 
 13  publishers            29078 non-null  object 
 14  package_groups        29078 non-null  object 
 15  platforms          

In [21]:
merged.drop(columns=['name_y', 'publisher','developers', 'genres','screenshots', 'supported_languages'], inplace=True)

In [22]:
merged.rename(columns={'name_x':'name', 'publishers':'publisher', 'average_forever':'avg_forever', 'average_2weeks':'avg_2_weeks'}, inplace=True)

In [23]:
merged = merged.drop_duplicates()

In [24]:
merged.isnull().sum()

type                    149
name                      1
appid                     0
required_age            149
is_free                 149
detailed_description    175
about_the_game          175
short_description       175
supported_languages     163
pc_requirements         149
mac_requirements        149
linux_requirements      149
publisher               149
package_groups          149
platforms               149
categories              714
screenshots             177
release_date            149
developer               197
positive                  0
negative                  0
userscore                 0
owners                    0
avg_forever               0
avg_2_weeks               0
median_forever            0
median_2weeks             0
price                    29
initialprice             22
discount                 22
languages                94
genre                   152
ccu                       0
tags                      0
dtype: int64

In [25]:
merged['type'].value_counts()

game    29078
Name: type, dtype: int64

In [26]:
def clean_objects(df):
    df = df[df['type'].notnull()]
    df = df[df['name'].notnull()]
    df = df[df['name'] != 'none']
    df = df.drop('type', axis=1)
    return df   
merged = clean_objects(merged)

In [27]:
merged['required_age'].value_counts().sort_index()

0.0       28430
1.0           1
3.0          10
4.0           2
5.0           1
6.0           1
7.0           8
10.0          3
11.0          4
12.0         72
13.0         21
14.0          4
15.0         39
16.0        141
17.0         47
18.0        288
20.0          1
1818.0        1
Name: required_age, dtype: int64

In [28]:
# Replace what is an obvious error to 18
merged = merged.replace(to_replace = 1818.0, value = 18)

In [29]:
mean = merged['required_age'].mean()
mode = merged['required_age'].mode()
print(f'Required Age mean:{mean}')
print(f'Required Age mode:{mode}')

Required Age mean:0.35203274403246887
Required Age mode:0    0.0
dtype: float64


In [30]:
merged['required_age'] = merged['required_age'].fillna(0)

https://www.internetmatters.org/resources/video-games-age-ratings-explained/
ratings in line with ersb rating system
everyone, 10+, teen, 18

In [31]:
# Code refrence from https://stackoverflow.com/questions/52753613/grouping-categorising-ages-column-in-python-pandas
bins = [-1, 0, 3, 7, 12, 16, 20]
labels = [0, 3, 7, 12, 16, 18]

merged['required_age'] = pd.cut(merged['required_age'], bins=bins, labels=labels)

In [32]:
merged['required_age'].value_counts()

0     28430
18      337
16      205
12       79
7        12
3        11
Name: required_age, dtype: int64

In [33]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29074 entries, 0 to 29226
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   name                  29074 non-null  object  
 1   appid                 29074 non-null  int64   
 2   required_age          29074 non-null  category
 3   is_free               29074 non-null  bool    
 4   detailed_description  29050 non-null  object  
 5   about_the_game        29050 non-null  object  
 6   short_description     29050 non-null  object  
 7   supported_languages   29061 non-null  object  
 8   pc_requirements       29074 non-null  object  
 9   mac_requirements      29074 non-null  object  
 10  linux_requirements    29074 non-null  object  
 11  publisher             29074 non-null  object  
 12  package_groups        29074 non-null  object  
 13  platforms             29074 non-null  object  
 14  categories            28510 non-null  object  
 15  sc

## Categories and genres

In [55]:
merged['genre'].value_counts()

Action, Indie                                                                   1971
Casual, Indie                                                                   1563
Action, Adventure, Indie                                                        1310
Adventure, Indie                                                                1228
Action, Casual, Indie                                                           1061
                                                                                ... 
Violent, Gore, Action, RPG                                                         1
Adventure, Casual, Indie, RPG, Sports                                              1
Free to Play, Massively Multiplayer, Sports                                        1
Sexual Content, Indie, RPG                                                         1
Sexual Content, Violent, Gore, Action, Casual, Free to Play, Indie, Strategy       1
Name: genre, Length: 1542, dtype: int64

In [57]:
merged['genre'].sample(5, random_state=5)

26509                 Action, Adventure, Indie
4422                                    Casual
26348    Casual, Indie, Strategy, Early Access
20430         Action, Adventure, Casual, Indie
8344                             Casual, Indie
Name: genre, dtype: object

In [58]:
merged.dropna(subset=['genre'], inplace=True)  

In [43]:
merged['categories'].value_counts()

[{'id': 2, 'description': 'Single-player'}]                                                                                                                                                                                                                                                                                                                                                                                                           6390
[{'id': 2, 'description': 'Single-player'}, {'id': 22, 'description': 'Steam Achievements'}]                                                                                                                                                                                                                                                                                                                                                          2478
[{'id': 2, 'description': 'Single-player'}, {'id': 22, 'description': 'Steam Achievements'}, {'id': 29, 'descripti

In [47]:
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html
merged['categories'].sample(5, random_state=5)

1790     [{'id': 2, 'description': 'Single-player'}, {'...
646            [{'id': 2, 'description': 'Single-player'}]
9286     [{'id': 2, 'description': 'Single-player'}, {'...
19       [{'id': 1, 'description': 'Multi-player'}, {'i...
26910    [{'id': 2, 'description': 'Single-player'}, {'...
Name: categories, dtype: object

In [50]:
merged[merged['categories'].isnull()].sample(5, random_state=5)

,name,appid,required_age,is_free,detailed_description,about_the_game,short_description,pc_requirements,mac_requirements,linux_requirements,...,avg_2_weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
8444,Aseprite,431730,0,False,Aseprite is a pixel art tool that lets you cre...,Aseprite is a pixel art tool that lets you cre...,Aseprite is a pixel-art tool to create 2D anim...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,...,0,4256,0,1499.0,1499.0,0.0,English,"Animation & Modeling, Design & Illustration, G...",798,"{'Pixel Graphics': 52, 'Design & Illustration'..."
17682,Grid Cartographer 4,684690,0,False,Grid Cartographer 4 is a powerful map creation...,Grid Cartographer 4 is a powerful map creation...,Grid Cartographer 4 is an intuitive map creati...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,...,0,0,0,4999.0,4999.0,0.0,English,Design & Illustration,0,"{'Design & Illustration': 24, 'Dungeon Crawler..."
8799,VirtualHere For Steam Link,440520,0,False,VirtualHere allows you to use the XBox One Wir...,VirtualHere allows you to use the XBox One Wir...,"Add support for XBox One wireless, steering wh...",{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,...,0,22,0,1399.0,1399.0,0.0,English,Utilities,13,{'Utilities': 22}
15668,Soundpad,629520,0,False,Play sounds in voice chats in high digital qua...,Play sounds in voice chats in high digital qua...,Play sounds in voice chats in high digital qua...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],...,3425,17405,3425,499.0,499.0,0.0,"English, German, French, Korean, Portuguese - ...","Audio Production, Utilities",1653,"{'Utilities': 29, 'Audio Production': 25}"
13616,Craftmas,572350,0,False,Craftmas is a virtual winter wonderland where ...,Craftmas is a virtual winter wonderland where ...,Create colorful 3D ornaments and decorate your...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],...,0,0,0,499.0,499.0,0.0,English,"Design & Illustration, Early Access",0,"{'Early Access': 21, 'Design & Illustration': 21}"


In [51]:
merged.dropna(subset=['categories'], inplace=True)  

In [59]:
merged.isnull().sum()

name                     0
appid                    0
required_age             0
is_free                  0
detailed_description    15
about_the_game          15
short_description       15
pc_requirements          0
mac_requirements         0
linux_requirements       0
publisher                0
package_groups           0
platforms                0
categories               0
release_date             0
developer               90
positive                 0
negative                 0
userscore                0
owners                   0
avg_forever              0
avg_2_weeks              0
median_forever           0
median_2weeks            0
price                    0
initialprice             0
discount                 0
languages                2
genre                    0
ccu                      0
tags                     0
dtype: int64

In [37]:
merged['package_groups'].head()

0    [{'name': 'default', 'title': 'Buy Counter-Str...
1    [{'name': 'default', 'title': 'Buy Team Fortre...
2    [{'name': 'default', 'title': 'Buy Day of Defe...
3    [{'name': 'default', 'title': 'Buy Deathmatch ...
4    [{'name': 'default', 'title': 'Buy Half-Life: ...
Name: package_groups, dtype: object

### Puplisher

In [61]:
merged['publisher'].value_counts()

['Big Fish Games']        231
['']                      210
['Strategy First']        137
['Ubisoft']               115
['Square Enix']           104
                         ... 
['RaveyLarge']              1
['Candescent Games']        1
['WeRVR Studio']            1
['NeutronStar.studio']      1
['Ambiera e.U.']            1
Name: publisher, Length: 14949, dtype: int64

In [74]:
merged[['developer', 'publisher']].iloc[40:50]

,developer,publisher
41,Arkane Studios,['Arkane Studios']
42,Valve,['Valve']
43,Reality Pump Studios,"['Topware Interactive', 'ACE']"
44,Reality Pump Studios,['Topware Interactive']
45,Arkane Studios,['Ubisoft']
46,id Software,['id Software']
47,RavenSoft / id Software,['Bethesda Softworks']
48,id Software,['Bethesda-Softworks']
49,id Software,['id Software']
50,id Software,['id Software']


In [77]:
merged.dropna(subset=['developer'], inplace=True)  

In [79]:
merged.dropna(subset=['languages'], inplace=True)  

In [80]:
merged.isnull().sum()

name                     0
appid                    0
required_age             0
is_free                  0
detailed_description    14
about_the_game          14
short_description       14
pc_requirements          0
mac_requirements         0
linux_requirements       0
publisher                0
package_groups           0
platforms                0
categories               0
release_date             0
developer                0
positive                 0
negative                 0
userscore                0
owners                   0
avg_forever              0
avg_2_weeks              0
median_forever           0
median_2weeks            0
price                    0
initialprice             0
discount                 0
languages                0
genre                    0
ccu                      0
tags                     0
dtype: int64

In [82]:
merged.to_csv('merged_stage2.csv', index=False)

In [ ]:
# Dropping collumns with more than 20% null values
df_app.drop(columns=['controller_support', 'dlc', 'fullgame', 'legal_notice', 'drm_notice', 'demos', 'metacritic', 'reviews', 'recommendations'], inplace=True)

In [ ]:
# Dropping columns that are not needed
df_app.drop(columns=['header_image', 'website', 'ext_user_account_notice', 'screenshots', 'movies', 'support_info', 'content_descriptors', 'background'], inplace=True)

In [ ]:
df_app['type'].value_counts()

In [ ]:
df_app['is_free'].value_counts()

In [ ]:
df_app['required_age'].value_counts()

In [ ]:
df_app.drop(columns=['type', 'is_free', 'required_age', 'supported_languages'],inplace=True)

In [ ]:
df_app.head()

In [ ]:
df_app.isnull().sum()

In [ ]:
df_app.to_csv('steam_app.csv', index=False)

### CLEANING steam

In [ ]:
df2 = pd.read_csv('./data/steam.csv')

In [ ]:
df2.head()

In [ ]:
df2.isnull().sum()

In [ ]:
df2.shape

### CLEANING steamspy_data

In [ ]:
df3 = pd.read_csv('./data/steamspy_data.csv')

In [ ]:
df3.head()

In [ ]:
df3.shape

In [ ]:
df3.isnull().sum()

In [ ]:
df3 = df3.drop(columns='score_rank')

In [ ]:
df3.to_csv('steamspy.csv', index=False)

### CLEANING MC15

In [ ]:
mc = pd.read_csv('./data/mc.csv')

In [ ]:
mc.head()

In [ ]:
mc.shape

In [ ]:
mc.isnull().sum()

In [ ]:
mc.to_csv('meta_critic.csv', index=False)

### CLEANING 200K